In [1]:
import numpy as np
import healpy as hp

In [2]:
import shtns

In [3]:
import math

In [4]:
import pylab as plt

In [5]:
import healpyTools as hpt

In [6]:
import zm_tools as zt

In [7]:
import kjones

In [8]:
import polarization_field_transform as pft

In [9]:
nside = 2**1
npix = hp.nside2npix(nside)
map1 = np.arange(npix)+1.

In [10]:
I = 10.
Q = 6.
U = -4.
V = 2.

const_coher_tensor = np.array([[I + Q,U - 1.j*V],[U + 1.j*V,I-Q]])

In [11]:
sky = const_coher_tensor.reshape(2,2,1).repeat(npix,2)

In [12]:
def t_hat_cart(t,p):
    """ Calculate the theta_hat vector at a given point (t,p) """
    thx = np.cos(t)*np.cos(p)
    thy = np.cos(t)*np.sin(p)
    thz = -np.sin(t)
    return np.stack((thx,thy,thz))

def p_hat_cart(t,p):
    """ Calculate the phi_hat vector at a given point (t,p) """
    phx = -np.sin(p)#/np.cos(t)
    phy = np.cos(p)#/np.cos(t)
    phz = np.zeros_like(p)
    return np.stack((phx,phy,phz))

def r_hat_cart(t,p):
    """ Calculate the r_hat vector at a given point (t,p) """
    rx = np.sin(t)*np.cos(p)
    ry = np.sin(t)*np.sin(p)
    rz = np.cos(t)
    return np.stack((rx,ry,rz))
def vrot(vector,theta,phi,rot):
    """ 
        Given a 3-D unit vector specifying the vector field at (theta,phi),
        rotate this field by rot and return the new vector at the new point 
    """
    R = hp.Rotator(rot=rot)
    rvector = R(vector)
    rtheta,rphi = R(theta,phi)
    return (rvector,rtheta,rphi)

In [13]:
def get_sphr_rotation_matrix(angle1,angle2,rotation_matrix):
    rotmat = np.asarray(rotation_matrix)
    
    vec0 = np.array([0,0,1])
    
    x = np.cos(phi)*np.sin(theta)
    y = np.sin(phi)*np.sin(theta)
    z = np.cos(theta)
    
    in_uvec = np.stack((x,y,z)) # the cartesian position vectors on the unit sphere.
    
    vec = np.einsum('ab...,b...->a...',rotmat,in_uvec)
    vec0 = np.einsum('ba,b->a',rotmat,vec0)
    
    sin_psi = vec0[1]*in_uvec[0,:] - vec0[0]*in_uvec[1,:]
    cos_psi = -vec0[2] + vec[2,:]*in_uvec[1,:]
    
    norm = sin_psi**2. + cos_psi**2. # this is effectively the determinant of the 
    
    sin_psi /= np.sqrt(norm)
    cos_psi /= np.sqrt(norm)
    
    return np.array([[cos_psi,sin_psi],[-sin_psi,cos_psi]])
#    return cos_psi,sin_psi

In [14]:
skyI = zt.stokes_project(sky,'I')
skyQ = zt.stokes_project(sky,'Q')
skyU = zt.stokes_project(sky,'U')
skyV = zt.stokes_project(sky,'V')

In [15]:
pixindex = np.arange(npix)
theta,phi = hp.pix2ang(nside,pixindex)
rotation_matrix = zt.rotation_matrix([0,1,0],np.pi/4.)
Q = skyQ
U = skyU

In [16]:
sphr_rot = get_sphr_rotation_matrix(theta,phi,rotation_matrix)

In [17]:
# rotsky = np.empty_like(sky)
rotsky = np.einsum('ab...,bc...,dc...->ad...',sphr_rot,sky,sphr_rot)

In [18]:
rotsky.shape

(2, 2, 48)

In [19]:
rotskyI = zt.stokes_project(rotsky,'I')
rotskyQ = zt.stokes_project(rotsky,'Q')
rotskyU = zt.stokes_project(rotsky,'U')
rotskyV = zt.stokes_project(rotsky,'V')

In [20]:
Qnew,Unew = zt.linear_pol_coord_transform(theta,phi,rotation_matrix,Q,U)

In [21]:
jones_data = kjones.get_hera_jones()

In [22]:
jones = jones_data['jones']
theta_j = jones_data['theta']
phi_j = jones_data['phi']

In [23]:
jones.shape

(2, 2, 65160)

In [27]:
lmax = 178
mmax = lmax
sh = shtns.sht(lmax,mmax)

In [30]:
nlat = 180
nphi = 360
grid_type = shtns.sht_reg_fast
polar_opt_threshold = 1.0e-10

In [31]:
sh.set_grid(nlat,nphi,flags=grid_type,polar_opt=polar_opt_threshold)

[180, 360]

In [32]:
el = sh.l

In [37]:
xy_beam = sh.spat_array()

In [38]:
xy_beam_ylm = sh.spec_array()

In [42]:
xy_beam.shape

(180, 360)

In [43]:
j_tx = jones[0,0,:]

In [59]:
theta_v, phi_v = np.meshgrid(np.arange(360)*np.pi/359.,np.arange(180)*np.pi/179.)

In [60]:
theta_v.shape

(180, 360)

In [61]:
phi_v.shape

(180, 360)

In [64]:
theta_v_f = theta_v.flatten()

In [65]:
theta_v_f.shape

(64800,)